# Belgrade traffic accidents analysis
This notebook is used for analysis of available data.

## Loading data
Firstly, loading data from MongoDB database

In [0]:
%scala
import spark.implicits._
import com.mongodb.spark._

val dfa = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("database", "belgrade_15-20").option("collection", "accidents").load()
val dfs = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("database", "belgrade_15-20").option("collection", "summary").load()
val dfo = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("database", "belgrade_15-20").option("collection", "observations").load()
val dfao = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("database", "belgrade_15-20").option("collection", "accidents_observations").load()

dfa.createOrReplaceTempView("accidents")
dfs.createOrReplaceTempView("summary")
dfo.createOrReplaceTempView("observations")
dfao.createOrReplaceTempView("accidents_observations")

import spark.implicits._
import com.mongodb.spark._
dfa: org.apache.spark.sql.DataFrame = [_id: struct<oid: string>, city: string ... 9 more fields]
dfs: org.apache.spark.sql.DataFrame = [_id: struct<oid: string>, average: double ... 10 more fields]
dfo: org.apache.spark.sql.DataFrame = [_id: struct<oid: string>, condition: string ... 4 more fields]
dfao: org.apache.spark.sql.DataFrame = [_id: struct<oid: string>, accident: timestamp ... 12 more fields]

In [0]:
dfa = spark.sql("select id, city, municipality, timestamp, day_time, latitude, longitude, outcome, participants, description from accidents")
dfs = spark.sql("select date(date), ROUND(((high - 32) * 5/9)) as high, ROUND(((high_historic - 32) * 5/9)) as high_historic, ROUND(((low - 32) * 5/9)) as low,ROUND(((low_historic - 32) * 5/9)) as low_historic, ROUND(((average - 32) * 5/9)) as average,ROUND(((average_historic - 32) * 5/9)) as average_historic,max_wind, pressure, sunrise, sunset from summary")
dfo = spark.sql("select timestamp, ROUND(((temperature - 32) * 5/9)) as temperature, humidity, pressure, condition from observations")
dfao = spark.sql("select id, municipality, accident, day_time, latitude, longitude, outcome, participants, observation, condition, ROUND(((temperature - 32) * 5/9)) as temperature, humidity, pressure from accidents_observations")
dfa.createOrReplaceTempView("accidents")
dfs.createOrReplaceTempView("summary")
dfo.createOrReplaceTempView("observations")
dfao.createOrReplaceTempView("acc_obs")

## Defining questions
- When do most accidents occur?
- Does temperature or weather condition affect frequency of traffic accidents?
- What is the most dangerous municipality for driving?
- What is the most dangerous municipality for pedestrians?

In [0]:
%sql
select municipality, count(id) as number from accidents
where date(timestamp) between '2016-01-01' and '2020-12-31'
group by municipality
order by number desc

municipality,number
NOVI BEOGRAD,12721
VOŽDOVAC,9861
PALILULA,8914
SAVSKI VENAC,7864
ČUKARICA,7786
ZEMUN,7337
ZVEZDARA,7224
VRAČAR,5121
STARI GRAD,4935
RAKOVICA,3210


In [0]:
%sql
drop table mun;
create table mun
select * from accidents
where year(timestamp)>=2016 and (municipality="NOVI BEOGRAD" or municipality="VOŽDOVAC" or municipality="PALILULA")

For simplicity sake, I decided to fokus on top three municipalities with highest number of accidents. Those are *Novi Beograd*, *Voždovac* and *Palilula*.

In [0]:
%sql
drop table accidents_by_municipality;
create table accidents_by_municipality
select municipality, year(timestamp) as year, month(timestamp) as month, count(id) as number from accidents
where date(timestamp) between '2016-01-01' and '2020-12-31'
group by municipality, year, month
order by year, month

In [0]:
%sql
select
  municipality, 
  sum(number) as sum,
  SUM(CASE WHEN month=1 THEN number END) AS `1`,
  SUM(CASE WHEN month=2 THEN number END) AS `2`,
  SUM(CASE WHEN month=3 THEN number END) AS `3`,
  SUM(CASE WHEN month=4 THEN number END) AS `4`,
  SUM(CASE WHEN month=5 THEN number END) AS `5`,
  SUM(CASE WHEN month=6 THEN number END) AS `6`,
  SUM(CASE WHEN month=7 THEN number END) AS `7`,
  SUM(CASE WHEN month=8 THEN number END) AS `8`,
  SUM(CASE WHEN month=9 THEN number END) AS `9`,
  SUM(CASE WHEN month=10 THEN number END) AS `10`,
  SUM(CASE WHEN month=11 THEN number END) AS `11`,
  SUM(CASE WHEN month=12 THEN number END) AS `12`
FROM accidents_by_municipality
GROUP BY municipality
ORDER BY sum DESC
LIMIT 3

municipality,sum,1,2,3,4,5,6,7,8,9,10,11,12
NOVI BEOGRAD,12721,1071,936,1068,982,1081,1093,935,972,1082,1225,1067,1209
VOŽDOVAC,9861,858,745,841,803,826,814,693,737,826,926,862,930
PALILULA,8914,770,686,709,676,717,746,716,716,759,825,759,835


In [0]:
%sql
select
  municipality, 
  year,
  SUM(CASE WHEN month=1 THEN number END) AS `1`,
  SUM(CASE WHEN month=2 THEN number END) AS `2`,
  SUM(CASE WHEN month=3 THEN number END) AS `3`,
  SUM(CASE WHEN month=4 THEN number END) AS `4`,
  SUM(CASE WHEN month=5 THEN number END) AS `5`,
  SUM(CASE WHEN month=6 THEN number END) AS `6`,
  SUM(CASE WHEN month=7 THEN number END) AS `7`,
  SUM(CASE WHEN month=8 THEN number END) AS `8`,
  SUM(CASE WHEN month=9 THEN number END) AS `9`,
  SUM(CASE WHEN month=10 THEN number END) AS `10`,
  SUM(CASE WHEN month=11 THEN number END) AS `11`,
  SUM(CASE WHEN month=12 THEN number END) AS `12`
FROM accidents_by_municipality
WHERE municipality="NOVI BEOGRAD"
GROUP BY municipality, year
ORDER BY year

municipality,year,1,2,3,4,5,6,7,8,9,10,11,12
NOVI BEOGRAD,2016,215,195,250,224,227,247,185,217,238,268,206,268
NOVI BEOGRAD,2017,206,154,227,202,239,243,195,200,249,263,297,250
NOVI BEOGRAD,2018,223,200,227,211,240,217,214,201,196,243,209,263
NOVI BEOGRAD,2019,215,202,227,251,222,191,179,202,208,226,232,252
NOVI BEOGRAD,2020,212,185,137,94,153,195,162,152,191,225,123,176


In [0]:
%sql
drop table accidents_in_lockdown;
create table accidents_in_lockdown
select municipality, 
       day,
       year(timestamp) as year,
       count(id) as number
from (select id,
       municipality,
       timestamp,
       day_time,
       weekday(timestamp) as day,
       outcome,
       participants
       from accidents)
where (date(timestamp) between '2019-04-01' and '2019-04-30' or date(timestamp) between '2020-04-01' and '2020-04-30')
group by municipality, day, year
order by municipality

In [0]:
%sql
select municipality,
       NVL(sum(case when year=2019 and day>=5 then number end),0) as `weekend2019`,
       NVL(sum(case when year=2020 and day>=5  then number end),0) as `weekend2020`,
       NVL(sum(case when year=2019 and day<=4  then number end),0) as `workdays2019`,
       NVL(sum(case when year=2020 and day<=4 then number end),0) as `workdays2020`,
       sum(case when year=2019 then number end) as `2019`,
       sum(case when year=2020 then number end) as `2020`
from accidents_in_lockdown
where municipality="NOVI BEOGRAD" or municipality="PALILULA" or municipality="VOŽDOVAC"
group by municipality

municipality,weekend2019,weekend2020,workdays2019,workdays2020,2019,2020
PALILULA,31,4,112,75,143,79
NOVI BEOGRAD,39,3,212,91,251,94
VOŽDOVAC,43,3,121,73,164,76


There's inconsistency in the number of accidents between 2020. and previous years. Bearing in mind that 2020 is the year of corona virus pandemic and that Serbian government periodically ordered lockdowns, I analyzed accidents during weekends of April, when citizens had total movement restrictions; results are drastic decrease of accidents. In conclusion, government measures had impanct on this anomaly.

In *Novi Beograd*, accidents occur more often than in *Voždovac* or *Palilula*.<br>
Number of accidents is much higher during october or december than other months, whereas in july is the lowest. For that reason, I check if there are some deviations in weater condition during those months.

In [0]:
%sql
select month(accident) as month, count(accident) as accidents, round(mean(temperature)) as mean
from acc_obs 
where year(accident)>=2016 and municipality="NOVI BEOGRAD"
group by month 
order by mean

month,accidents,mean
1,1066,1.0
12,1207,4.0
2,936,6.0
11,1055,9.0
3,1066,10.0
10,1215,15.0
4,982,16.0
5,1072,19.0
9,1080,21.0
6,1091,24.0


In [0]:
%sql
select condition, count(condition) as accidents
from acc_obs
where not (month(accident)=10 or month(accident)=12) and year(accident)>=2016 and municipality="NOVI BEOGRAD"
group by condition
order by condition

condition,accidents
Cloudy,75
Drizzle,7
Fair,6759
Fair / Windy,110
Fog,216
Heavy Snow,4
Light Drizzle,35
Light Rain,445
Light Rain / Windy,24
Light Rain Shower,61


In [0]:
%sql
select condition, count(condition) as accidents
from acc_obs
where (month(accident)=10 or month(accident)=12) and year(accident)>=2016 and municipality="NOVI BEOGRAD"
group by condition
order by condition

condition,accidents
Cloudy,51
Drizzle,1
Fair,1282
Fair / Windy,43
Fog,75
Heavy Rain Shower / Windy,1
Heavy Snow,8
Light Drizzle,8
Light Rain,151
Light Rain / Windy,25


In [0]:
%sql
select condition, count(condition) as occured
from observations
where (month(timestamp)=10 or month(timestamp)=12) and year(timestamp)>=2016
group by condition

condition,occured
Light Rain Shower / Windy,3
Shallow Fog,14
Light Snow / Windy,1
Cloudy,390
Wintry Mix,12
Rain Shower,1
Light Rain with Thunder,1
Patches of Fog,9
Drizzle,12
T-Storm,1


In [0]:
%sql
select a.condition, count(a.condition) as accidents
from acc_obs as a
where (month(accident)=10 or month(accident)=12) and year(accident)>=2016 and municipality="NOVI BEOGRAD"
group by a.condition
order by a.condition

condition,accidents
Cloudy,51
Drizzle,1
Fair,1282
Fair / Windy,43
Fog,75
Heavy Rain Shower / Windy,1
Heavy Snow,8
Light Drizzle,8
Light Rain,151
Light Rain / Windy,25


Number of accidents in *Novi Beograd* by weather conditions during oktober and december in oppose to other months:<br>
Windy 43-172<br>
Snow 62-192<br>
Rain 204-690<br>
Fog 358-692<br>
Cloudy 548-1727<br>
Fair 1282-6759<br>
<br>
Finding that there is no significant deviations in temperature or number of accidents by weather conditions, I conclude that weather has no important impact on frequency of accidents.

In [0]:
%sql
select a.municipality, count(a.outcome) as accidents, i.num as injuries, ROUND(count(a.outcome)/i.num,2) as iratio, 
                                                      d.num as deaths, ROUND(count(a.outcome)/d.num,2) as dratio
from accidents as a
inner join (select municipality, count(outcome) as num
            from accidents
            where outcome="death" and year(timestamp)>=2016
            group by municipality) as d
on a.municipality=d.municipality
inner join (select municipality, count(outcome) as num
            from accidents
            where outcome="injury" and year(timestamp)>=2016
            group by municipality) as i
on a.municipality=i.municipality
where year(a.timestamp)>=2016
group by a.municipality, d.num, i.num
order by dratio

municipality,accidents,injuries,iratio,deaths,dratio
LAZAREVAC,1507,654,2.3,46,32.76
BARAJEVO,885,310,2.85,20,44.25
SURČIN,2118,619,3.42,37,57.24
SOPOT,446,199,2.24,7,63.71
MLADENOVAC,1240,553,2.24,19,65.26
OBRENOVAC,1977,888,2.23,28,70.61
GROCKA,2137,800,2.67,27,79.15
ČUKARICA,7786,2108,3.69,59,131.97
ZEMUN,7337,2168,3.38,45,163.04
RAKOVICA,3210,763,4.21,18,178.33


In [0]:
%sql

select day_time, count(id) as number
from accidents
where year(timestamp)>=2016 and municipality="LAZAREVAC" and outcome="death"
group by day_time
order by number desc

day_time,number
afternoon,12
evening,10
early morning,6
midday,6
after midnight,5
early evening,4
morning,3


In [0]:
%sql
select a.municipality, count(a.outcome) as accidents, i.num as injuries, ROUND(count(a.outcome)/i.num,2) as iratio, 
                                                      d.num as deaths, ROUND(count(a.outcome)/d.num,2) as dratio
from accidents as a
inner join (select municipality, count(outcome) as num
            from accidents
            where outcome="death" and year(timestamp)>=2016 and day_time="afternoon"
            group by municipality) as d
on a.municipality=d.municipality
inner join (select municipality, count(outcome) as num
            from accidents
            where outcome="injury" and year(timestamp)>=2016 and day_time="afternoon"
            group by municipality) as i
on a.municipality=i.municipality
where year(a.timestamp)>=2016 and day_time="afternoon"
group by a.municipality, d.num, i.num
order by dratio

municipality,accidents,injuries,iratio,deaths,dratio
LAZAREVAC,392,183,2.14,12,32.67
BARAJEVO,218,81,2.69,6,36.33
MLADENOVAC,343,168,2.04,8,42.88
OBRENOVAC,521,261,2.0,9,57.89
SOPOT,124,57,2.18,2,62.0
GROCKA,532,206,2.58,5,106.4
ČUKARICA,2068,576,3.59,16,129.25
SURČIN,558,171,3.26,4,139.5
VOŽDOVAC,2684,689,3.9,13,206.46
ZEMUN,2019,613,3.29,8,252.38


In [0]:
%sql
drop table acc_daytime;
create table acc_daytime
select municipality, day_time, count(id) as num
from accidents
where year(timestamp)>=2016
group by municipality, day_time
order by municipality, day_time

In [0]:
%sql
select a.municipality, a.day_time, t.max
from acc_daytime as a
inner join  (select municipality, max(num) as max
            from acc_daytime
            group by municipality) as t
on a.municipality=t.municipality and a.num=t.max
order by a.municipality

municipality,day_time,max
BARAJEVO,afternoon,218
GROCKA,afternoon,532
LAZAREVAC,afternoon,392
MLADENOVAC,afternoon,343
NOVI BEOGRAD,afternoon,3974
OBRENOVAC,afternoon,521
PALILULA,afternoon,2445
RAKOVICA,afternoon,823
SAVSKI VENAC,afternoon,2353
SOPOT,afternoon,124


In [0]:
%sql
select a.municipality, count(a.outcome) as accidents, i.num as afternoon, ROUND(count(a.outcome)/i.num,2) as ratio
from accidents as a
inner join (select municipality, count(id) as num
            from accidents
            where day_time="afternoon" and year(timestamp)>=2016
            group by municipality) as i
on a.municipality=i.municipality
where year(a.timestamp)>=2016
group by a.municipality, i.num
order by ratio

municipality,accidents,afternoon,ratio
NOVI BEOGRAD,12721,3974,3.2
SAVSKI VENAC,7864,2353,3.34
STARI GRAD,4935,1395,3.54
VRAČAR,5121,1430,3.58
SOPOT,446,124,3.6
MLADENOVAC,1240,343,3.62
ZEMUN,7337,2019,3.63
PALILULA,8914,2445,3.65
VOŽDOVAC,9861,2684,3.67
ČUKARICA,7786,2068,3.76


In [0]:
%sql
select a.municipality, count(a.outcome) as accidents, p.num as `pedestrian involved`, ROUND(count(a.outcome)/p.num) as ratio,
                                                      d.num as dnumber, ROUND(p.num/d.num) as dratio
from accidents as a
inner join (select municipality, count(outcome) as num
            from accidents
            where participants="pedestrian" and year(timestamp)>=2016
            group by municipality) as p
on a.municipality=p.municipality
inner join (select municipality, count(outcome) as num
            from accidents
            where participants="pedestrian" and year(timestamp)>=2016 and outcome="death"
            group by municipality) as d
on a.municipality=d.municipality
where year(a.timestamp)>=2016
group by a.municipality, p.num, d.num
order by dratio, ratio

municipality,accidents,pedestrian involved,ratio,dnumber,dratio
BARAJEVO,885,71,12.0,8,9.0
SURČIN,2118,88,24.0,10,9.0
LAZAREVAC,1507,109,14.0,10,11.0
SOPOT,446,34,13.0,2,17.0
ČUKARICA,7786,516,15.0,31,17.0
GROCKA,2137,165,13.0,9,18.0
ZEMUN,7337,464,16.0,24,19.0
NOVI BEOGRAD,12721,679,19.0,28,24.0
MLADENOVAC,1240,116,11.0,4,29.0
STARI GRAD,4935,272,18.0,9,30.0


Surprisingly, municipalties with the least number of accidents are much more risky; *Lazarevac* (1507) and *Barajevo* (885) are the most risky, where every 33td and 44th accident respectively, ends with death. *Barajevo* is also the most dangerous for pedestrians where every 12th accident includes pedestrian and every 111th results in death.<br>
On the other hand, I consider *Vracar* as safest municipality, where in 5121 accidents, only 8 resulted in death.<br>
Also, afternoon (14-18h) is the period of day with the highest number of accidents and the most difficult outcomes.